In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "travel_server": {
                "transport": "streamable_http",
                "url": "https://mcp.kiwi.com"
            }
    }
)

tools = await client.get_tools()

  + Exception Group Traceback (most recent call last):
  |   File "/home/user/langchain/1-langchain_intro/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3699, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "/tmp/ipykernel_52467/1543074601.py", line 12, in <module>
  |     tools = await client.get_tools()
  |             ^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/home/user/langchain/1-langchain_intro/.venv/lib/python3.12/site-packages/langchain_mcp_adapters/client.py", line 197, in get_tools
  |     tools_list = await asyncio.gather(*load_mcp_tool_tasks)
  |                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/home/user/langchain/1-langchain_intro/.venv/lib/python3.12/site-packages/langchain_mcp_adapters/tools.py", line 478, in load_mcp_tools
  |     async with create_session(
  |   File "/usr/lib/python3.12/contextlib.py", line 231, in __aexit__
  |     await self.gen.athrow(value)
  |   File "/home/user

In [ ]:
from langchain_ollama import ChatOllama

model = ChatOllama(model="gpt-oss:20b", temperature=0)

In [ ]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

system_prompt = """You are a travel agent. 
IMPORTANT: When using the search-flight tool, you MUST use the following rules:
1. Dates must be in 'dd/mm/yyyy' format (e.g., 31/03/2026).
2. All dates must be in the future.
3. If the user asks for a one-way flight, ensure you still provide the correct parameters required by the tool.
No follow up questions."""

agent = create_agent(
    model=model,
    tools=tools,
    checkpointer=InMemorySaver(),
    system_prompt=system_prompt
)

In [ ]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Create an effective prompt template for getting flights")]},
    config
    )

In [ ]:
from pprint import pprint

pprint(response)

In [ ]:
print(response["messages"][-1].content)

In [ ]:
user_prompt = """
Use the json below to find flights:

```json
{
  "cabinClass": "M",
  "curr": "USD",
  "departureDate": "11/07/2026",
  "departureDateFlexRange": 1,
  "flyFrom": "Los Angeles",
  "flyTo": "Tokyo",
  "locale": "en",
  "passengers": {"adults": 2, "children": 0, "infants": 0},
  "returnDate": "08/08/2026",
  "returnDateFlexRange": 1,
  "sort": "price"
}
```
"""

response = await agent.ainvoke(
    {"messages": [HumanMessage(content=user_prompt)]},
    config
    )

pprint(response)

In [ ]:
print(response["messages"][-1].content)

In [ ]:
user_prompt = """
Find me a one-way economy flight from Paris CDG to Barcelona BCN on July 15, 2026. 
I'm flexible by one day and want the cheapest price in EUR.
"""

response = await agent.ainvoke(
    {"messages": [HumanMessage(content=user_prompt)]},
    config
    )

pprint(response)

In [ ]:
print(response["messages"][-1].content)

In [ ]:
user_prompt = """
I'm planning a summer trip. Can you look for flights for 2 adults and 1 child 
from New York JFK to Tokyo NRT? We want to leave on August 1st, 2026, and come 
back ten days later. Business class and sort by the shortest duration.
"""

response = await agent.ainvoke(
    {"messages": [HumanMessage(content=user_prompt)]},
    config
    )

pprint(response)

In [ ]:
print(response["messages"][-1].content)